In [ ]:
# Imports (global)
import os
import re
import glob
import pathlib
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.cluster.hierarchy import linkage, leaves_list

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import TwoSlopeNorm

from statsmodels.tsa.stattools import adfuller
from statsmodels.stats.diagnostic import acorr_ljungbox

# Output dir: <notebook_folder>/figures
try:
    import ipynbname  # optional

    _NB_DIR = Path(ipynbname.path()).parent
except Exception:
    _NB_DIR = Path.cwd()

OUT_DIR = _NB_DIR / "figures"
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("Notebook dir:", _NB_DIR)
print("Figures dir:", OUT_DIR)


# Data profiling – missingness & correlations
This notebook builds a panel from raw Excel files and analyzes missingness, correlations, and relationships to industrial production (IP).


In [ ]:
## 1) Helpers: date parsing & missingness patterns
def _to_month_start(ts):
    if pd.isna(ts): return pd.NaT
    ts = pd.Timestamp(ts)
    return pd.Timestamp(ts.year, ts.month, 1)

DOT_DDMMYY = re.compile(r'^\s*\d{2}\.\d{2}\.(\d{2}|\d{4})\s*$')

def parse_month(x):
    if pd.isna(x): return pd.NaT
    if isinstance(x, (int, float)) and not np.isnan(x):
        return _to_month_start(pd.to_datetime(x, unit='d', origin='1899-12-30', errors='coerce'))
    s = str(x).strip()
    if DOT_DDMMYY.fullmatch(s):
        return _to_month_start(pd.to_datetime(s, dayfirst=True, errors='coerce'))
    if re.fullmatch(r"\d{2}[./]\d{4}", s):
        return _to_month_start(pd.to_datetime(s.replace(".", "/"), format="%m/%Y", errors="coerce"))
    if re.fullmatch(r"\d{4}[./]\d{2}", s):
        y, m = re.split(r"[./]", s); return pd.Timestamp(int(y), int(m), 1)
    if re.fullmatch(r"\d{4}", s):
        return pd.Timestamp(int(s), 1, 1)
    dt = pd.to_datetime(s, errors="coerce", dayfirst=True)
    return _to_month_start(dt) if pd.notna(dt) else pd.NaT

def missing_pattern_report(df: pd.DataFrame):
    lead_flags, trail_flags, mid_flags, any_flags, only_flags = [], [], [], [], []

    for c in df.columns:
        a = df[c].isna().to_numpy()

        if not a.any():
            lead_flags.append(False)
            trail_flags.append(False)
            mid_flags.append(False)
            any_flags.append(False)
            only_flags.append(False)
            continue

        if a.all():
            only_flags.append(True)
        else:
            only_flags.append(False)

        lead = 0
        for v in a:
            if v: lead += 1
            else: break

        trail = 0
        for v in a[::-1]:
            if v: trail += 1
            else: break

        mid = a[lead:len(a)-trail].any() if (lead+trail) < len(a) else False

        lead_flags.append(lead > 0)
        trail_flags.append(trail > 0)
        mid_flags.append(mid)
        any_flags.append(True)

    lead_flags = np.array(lead_flags)
    trail_flags = np.array(trail_flags)
    mid_flags = np.array(mid_flags)
    any_flags = np.array(any_flags)
    only_flags = np.array(only_flags)

    return {
        "completely_NA": int(np.sum(only_flags)),
        "leading_only": int(np.sum(lead_flags & ~trail_flags & ~mid_flags & ~only_flags)),
        "trailing_only": int(np.sum(trail_flags & ~lead_flags & ~mid_flags & ~only_flags)),
        "mid_only": int(np.sum(mid_flags & ~lead_flags & ~trail_flags & ~only_flags)),
        "leading_and_trailing": int(np.sum(lead_flags & trail_flags & ~mid_flags & ~only_flags)),
        "leading_and_mid": int(np.sum(lead_flags & mid_flags & ~trail_flags & ~only_flags)),
        "trailing_and_mid": int(np.sum(trail_flags & mid_flags & ~lead_flags & ~only_flags)),
        "leading_trailing_mid": int(np.sum(lead_flags & trail_flags & mid_flags & ~only_flags)),
        "any_NA": int(np.sum(any_flags)),
    }

## 2) Build panel from raw Excel files

In [ ]:
DATA_DIR = pathlib.Path("../../data/raw/features")
files = sorted(map(pathlib.Path, glob.glob(str(DATA_DIR / "bdi1a_*.xlsx"))))
if not files:
    raise FileNotFoundError("No bdi1a_*.xlsx files found")

long_frames = []
for p in files:
    df = pd.read_excel(p, skiprows=2, engine="openpyxl")
    date_col = df.columns[0]
    df = df.rename(columns={date_col: "date"})
    df["date"] = df["date"].map(parse_month)
    long_frames.append(df.melt(id_vars="date", var_name="title_raw", value_name="value"))

panel_long = pd.concat(long_frames, ignore_index=True).dropna(subset=["date"])

full_df = panel_long.pivot_table(
    index="date", columns="title_raw", values="value", aggfunc="last", dropna=False
)
full_df.index = pd.to_datetime(full_df.index, errors="coerce").to_period("M").to_timestamp()
full_df = full_df[~full_df.index.duplicated(keep="last")]

print("Full merged shape:", full_df.shape)

report = missing_pattern_report(full_df)
print("Missingness report (columns):")
for k, v in report.items():
    print(f"{k}: {v}")

## 3) Paths & global settings

In [ ]:
features_file = "../../data/processed/features.csv"  # all features incl. missing
target_file   = "../../data/processed/target.csv"    # IP (level + change?)

industry_whitelist = [
    # "Verarbeitendes_Gewerbe",
    # "Herstellung_von_Investitionsgütern",
]

rm_window = 12  # months

## 4) Target diagnostics: stationarity tests

In [ ]:
df = pd.read_csv(target_file, parse_dates=["date"])
mom = df["IP_change"].dropna()
print("Mean:", mom.mean())
print("Variance:", mom.var())

def adf_test(series, title=''):
    result = adfuller(series.dropna())
    print(f"--- {title} ---")
    print("ADF statistic:", result[0])
    print("p-value:", result[1])
    for key, value in result[4].items():
        print(f"Critical value {key}: {value}")
    print()

adf_test(df["IP"], "IP (Level)")
adf_test(df["IP_change"], "IP_change (MoM)")
adf_test(df["IP_yoy"], "IP_yoy (YoY)")

## 5) Feature naming: canonical question types & splitting

In [ ]:
QUESTION_CANON = [
    "auftragsbestand_beurteilung",
    "auftragsbestand_beurteilung_export",
    "auftragsbestand_gegen_vormonat",
    "beschaeftigtenerwartungen",
    "exporterwartungen",
    "fertigwarenlager_beurteilung",
    "geschaeftsklima",
    "geschaeftslage_beurteilung",
    "geschaeftslage_erwartungen",
    "nachfrage_gegen_vormonat",
    "preise_gegen_vormonat",
    "preiserwartungen",
    "produktion_gegen_vormonat",
    "produktionsplaene",
    "produktivitaetserwartungen"
]

def _slug(s: str) -> str:
    s = s.lower()
    s = (s.replace("ä","ae").replace("ö","oe").replace("ü","ue").replace("ß","ss")
           .replace(" ", "_").replace("-", "_"))
    s = re.sub(r"[^a-z0-9_\.]+", "", s)
    s = re.sub(r"\.+", ".", s)
    s = re.sub(r"_+", "_", s)
    return s.strip("._")

def split_norm(col: str):
    s = _slug(col)
    for q in QUESTION_CANON:
        if s.endswith("." + q):
            ind = s[:-(len(q)+1)].strip("._")
            return ind, q
        if s.endswith(q):
            before = s[:-(len(q))]
            if before.endswith("."):
                ind = before[:-1].strip("._")
                return ind, q
    if "." in s:
        ind, qraw = s.rsplit(".", 1)
        q = re.sub(r"^(?:\d+\.|n\.?g\.?|ng\.?)+", "", qraw)
        q = q.strip("._")
        return ind.strip("._"), q
    return "", s

## 6) Load processed features/targets and align time index

In [ ]:
def _read_timeseries_csv(path):
    df = pd.read_csv(path)
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
        df = df.set_index('date')
    else:
        df.iloc[:,0] = pd.to_datetime(df.iloc[:,0])
        df = df.set_index(df.columns[0])
    return df.sort_index()

X_full = _read_timeseries_csv(features_file)
y_df   = _read_timeseries_csv(target_file)

X_original = _read_timeseries_csv("../../data/processed/features.csv")

X_original_NA_cutted = X_original.copy()
while X_original_NA_cutted.iloc[0].isna().any():
    X_original_NA_cutted = X_original_NA_cutted.iloc[1:]
while X_original_NA_cutted.iloc[-1].isna().any():
    X_original_NA_cutted = X_original_NA_cutted.iloc[:-1]

y_cols = list(y_df.columns)
if len(y_cols) >= 2:
    y_level  = y_df.iloc[:,0].astype(float)
    y_change = y_df.iloc[:,1].astype(float)
else:
    y_level  = y_df.iloc[:,0].astype(float)
    y_change = y_level.diff()

if industry_whitelist:
    wl = set(industry_whitelist)
    keep = [c for c in X_full.columns if c.split('.',1)[0] in wl]
    X_full = X_full[keep]
    X_original = X_original[keep]
    X_original_NA_cutted = X_original_NA_cutted[keep]
    print(f"Filtered columns: {len(keep)}")

idx = X_full.index.intersection(y_df.index)
X_full = X_full.loc[idx]
y_level = y_level.loc[idx]
y_change = y_change.loc[idx]

print("X_full", X_full.shape, "y_change", y_change.shape)
print("X_original_NA_cutted", X_original_NA_cutted.shape)


## 7) Missingness summary on X_full

In [ ]:
def missing_pattern_report(df: pd.DataFrame):
    lead_flags, trail_flags, mid_flags, any_flags = [], [], [], []

    for c in df.columns:
        a = df[c].isna().to_numpy()
        if not a.any():
            lead_flags.append(False)
            trail_flags.append(False)
            mid_flags.append(False)
            any_flags.append(False)
            continue

        lead = 0
        for v in a:
            if v: lead += 1
            else: break

        trail = 0
        for v in a[::-1]:
            if v: trail += 1
            else: break

        mid = a[lead:len(a)-trail].any() if (lead+trail) < len(a) else False

        lead_flags.append(lead > 0)
        trail_flags.append(trail > 0)
        mid_flags.append(mid)
        any_flags.append(True)

    lead_flags = np.array(lead_flags)
    trail_flags = np.array(trail_flags)
    mid_flags = np.array(mid_flags)
    any_flags = np.array(any_flags)

    report = {
        "leading_only": int(np.sum(lead_flags & ~trail_flags & ~mid_flags)),
        "trailing_only": int(np.sum(trail_flags & ~lead_flags & ~mid_flags)),
        "mid_only": int(np.sum(mid_flags & ~lead_flags & ~trail_flags)),
        "leading_and_trailing": int(np.sum(lead_flags & trail_flags & ~mid_flags)),
        "leading_and_mid": int(np.sum(lead_flags & mid_flags & ~trail_flags)),
        "trailing_and_mid": int(np.sum(trail_flags & mid_flags & ~lead_flags)),
        "leading_trailing_mid": int(np.sum(lead_flags & trail_flags & mid_flags)),
        "any_NA": int(np.sum(any_flags)),
    }
    return report

report = missing_pattern_report(X_full)
for k,v in report.items():
    print(f"{k}: {v}")

## 8) Correlation structure: clustered heatmap + extreme pairs

In [ ]:
corr = X_full.corr()

linkage_matrix = linkage(corr, method="ward")
ordered_idx = leaves_list(linkage_matrix)
corr_sorted = corr.iloc[ordered_idx, ordered_idx]

plt.figure(figsize=(10,8))
sns.heatmap(
    corr_sorted,
    cmap="RdBu_r",
    center=0,
    vmin=-1, vmax=1,
    cbar=True,
    xticklabels=False,
    yticklabels=False
)
plt.axis("off")

save_path = OUT_DIR / "corr_heatmap_clustered.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
print(f"Saved to: {save_path}")

plt.show()

mask = np.tril(np.ones(corr.shape, dtype=bool))
corr_unstacked = corr.where(~mask).unstack().dropna()

max_pair = corr_unstacked.idxmax(), corr_unstacked.max()
min_pair = corr_unstacked.idxmin(), corr_unstacked.min()

print("Strongest positive correlation:", max_pair)
print("Strongest negative correlation:", min_pair)

high_pos = (corr_unstacked > 0.99).sum()
high_neg = (corr_unstacked < -0.9).sum()

print(f"Count correlations > 0.9: {high_pos}")
print(f"Count correlations < -0.9: {high_neg}")

## 9) Correlation distribution (pairwise)

In [ ]:
corr_full = X_full.corr()
corr_cut  = X_original_NA_cutted.corr()

mask = np.tril(np.ones(corr_full.shape, dtype=bool))
vals_full = corr_full.where(~mask).stack().values
vals_cut  = corr_cut.where(~mask).stack().values

plt.figure(figsize=(8, 4))

sns.kdeplot(
    vals_full,
    fill=True,
    bw_adjust=0.6,
    label="ifo features without missing values"
)
sns.kdeplot(
    vals_cut,
    fill=True,
    bw_adjust=0.6,
    alpha=0.35,
    label="Full panel (common window)"
)

plt.xlim(-1, 1)
plt.xlabel("Pairwise Pearson Correlation")
plt.ylabel("Kernel density")
plt.title("Distribution of pairwise correlations across ifo Business Survey features")
plt.legend(frameon=True, title="Feature set")
plt.grid(True, alpha=0.25)
plt.tight_layout()

save_path = OUT_DIR / "correlation_distribution.png"
plt.savefig(save_path, dpi=300, bbox_inches="tight")
print(f"Saved to: {save_path}")

plt.show()

## 10) Missingness flags per column (lead/mid/trail)

In [ ]:
def missing_pattern_flags(s: pd.Series):
    a = s.isna().to_numpy()
    if not a.any():
        return False, False, False, False  # has, lead, mid, trail
    lead = 0
    for v in a:
        if v: lead += 1
        else: break
    trail = 0
    for v in a[::-1]:
        if v: trail += 1
        else: break
    has = True
    if lead + trail >= len(a):
        mid = False
    else:
        mid = a[lead:len(a)-trail].any()
    return has, (lead>0), mid, (trail>0)

flags = np.array([missing_pattern_flags(X_full[c]) for c in X_full.columns], dtype=bool)
if flags.size == 0:
    raise ValueError("No columns in X_full.")

has_miss = flags[:,0]
lead = flags[:,1]
mid  = flags[:,2]
trail= flags[:,3]

n = X_full.shape[1]
pct = lambda k: 100.0 * k / n

print(f"% with ≥1 missing:  {pct(has_miss.sum()):.1f}")
print(f"% missing at start: {pct(lead.sum()):.1f}")
print(f"% missing in middle:{pct(mid.sum()):.1f}")
print(f"% missing at end:   {pct(trail.sum()):.1f}")
print("-- combinations --")
print(f"% start & end:      {pct((lead & trail).sum()):.1f}")
print(f"% start & middle:   {pct((lead & mid).sum()):.1f}")
print(f"% end & middle:     {pct((trail & mid).sum()):.1f}")

## 11) Leading NAs over time: still-missing share (abs/rel)

In [ ]:
lead_cols = [c for c, l in zip(X_full.columns, lead) if l]
if len(lead_cols) == 0:
    print("No columns with leading NAs")
else:
    fvi = {}
    arr_index = np.arange(len(X_full.index))
    for c in lead_cols:
        a = X_full[c].isna().to_numpy()
        lv = 0
        for v in a:
            if v:
                lv += 1
            else:
                break
        fvi[c] = lv

    fvi_vals = np.array(list(fvi.values()))

    abs_curve = []
    for t in arr_index:
        still_missing = (fvi_vals > t).sum()
        abs_curve.append(still_missing / len(lead_cols))

    rel_curve = []
    denom = X_full.shape[1]
    for t in arr_index:
        still_missing = (fvi_vals > t).sum()
        rel_curve.append(still_missing / denom)

    plt.figure()
    plt.plot(X_full.index, abs_curve)
    plt.xlabel("Year")
    plt.ylabel("Share")
    plt.title("Leading NAs: Share still missing within leading-NA group")
    plt.tight_layout()

    save_path1 = OUT_DIR / "leading_NA_share_abs.png"
    plt.savefig(save_path1, dpi=300, bbox_inches="tight")
    print(f"Saved to: {save_path1}")

    plt.show()

    plt.figure()
    plt.plot(X_full.index, rel_curve)
    plt.xlabel("Year")
    plt.ylabel("Share (relative to all columns)")
    plt.title("Leading NAs: Share still missing within all time series")
    plt.tight_layout()

    save_path2 = OUT_DIR / "leading_NA_share_rel.png"
    plt.savefig(save_path2, dpi=300, bbox_inches="tight")
    print(f"Saved to: {save_path2}")

    plt.show()

jump_counts = pd.Series(fvi_vals).value_counts().sort_index()
for pos, count in jump_counts.items():
    print(f"{count} series start at {X_full.index[pos].date()}")

## 12) Missing rate per question + leading-NA share per question

In [ ]:
mapping = {
    "geschaeftsklima": "Business Climate",
    "geschaeftslage_beurteilung": "Business Situation Assessment",
    "geschaeftslage_erwartungen": "Business Expectations",
    "beschaeftigungserwartung": "Employment Expectations",
    "fertigwarenlager_beurteilung": "Assessment of Stock of Finished Goods",
    "auftragsbestand_beurteilung": "Assessment of Order Backlog",
    "auftragsbestand_beurteilung_export": "Assessment of Export Order Backlog",
    "nachfrage_gegen_vormonat": "Demand Compared to Previous Month",
    "auftragsbestand_gegen_vormonat": "Order Backlog Compared to Previous Month",
    "produktion_gegen_vormonat": "Production Compared to Previous Month",
    "preise_gegen_vormonat": "Prices Compared to Previous Month",
    "produktionsplaene": "Production Plans",
    "preiserwartungen": "Price Expectations",
    "exporterwartungen": "Export Expectations",
    "beschaeftigtenerwartungen": "Employment Expectations",
    "produktivitaetserwartungen": "Productivity Expectations"
}

def missing_pattern_flags(s: pd.Series):
    a = s.isna().to_numpy()
    if not a.any(): return False, False, False, False
    lead = 0
    for v in a:
        if v: lead += 1
        else: break
    trail = 0
    for v in a[::-1]:
        if v: trail += 1
        else: break
    mid = a[lead:len(a)-trail].any() if (lead+trail) < len(a) else False
    return True, (lead>0), mid, (trail>0)

by_qnorm = {}
for c in X_full.columns:
    _, qn = split_norm(c)
    if qn:
        by_qnorm.setdefault(qn, []).append(c)

q_missing_rate, q_leading_share = {}, {}
for qn, cols in by_qnorm.items():
    sub = X_full[cols]
    miss_rate = sub.isna().sum().sum() / (sub.shape[0]*sub.shape[1])
    lead_mask = [missing_pattern_flags(X_full[c])[1] for c in cols]
    q_missing_rate[qn] = miss_rate
    q_leading_share[qn] = (np.sum(lead_mask) / len(cols)) if cols else 0.0

qs = list(q_missing_rate.keys())

vals = [q_missing_rate[q]*100 for q in qs]
labels = [mapping.get(q, q) for q in qs]

plt.figure(figsize=(14,6))
plt.bar(labels, vals)
plt.xticks(rotation=45, ha="right")
plt.ylabel("% Missing (all cells)")
plt.title("Missing rate per question (normalized)")
plt.tight_layout()

save_path1 = OUT_DIR / "missing_rate_per_question.png"
plt.savefig(save_path1, dpi=300, bbox_inches='tight')
print(f"Saved to: {save_path1}")

plt.show()

vals2 = [q_leading_share[q]*100 for q in qs]

plt.figure(figsize=(14,6))
plt.bar(labels, vals2)
plt.xticks(rotation=45, ha="right")
plt.ylabel("% Columns with initial NA")
plt.title("Share of Leading NAs per question")
plt.tight_layout()

save_path2 = OUT_DIR / "leading_NA_share_per_question.png"
plt.savefig(save_path2, dpi=300, bbox_inches='tight')
print(f"Saved to: {save_path2}")

plt.show()

## 13) Leading & trailing NAs over time: still-missing share (abs/rel)

In [ ]:
lead_cols = [c for c, l in zip(X_full.columns, lead) if l]
if len(lead_cols) == 0:
    print("No columns with leading NAs")
else:
    fvi_lead, fvi_trail = {}, {}
    arr_index = np.arange(len(X_full.index))

    for c in lead_cols:
        a = X_full[c].isna().to_numpy()

        lv = 0
        for v in a:
            if v: lv += 1
            else: break
        fvi_lead[c] = lv

        tv = 0
        for v in a[::-1]:
            if v: tv += 1
            else: break
        fvi_trail[c] = tv

    fvi_lead_vals = np.array(list(fvi_lead.values()))
    fvi_trail_vals = np.array(list(fvi_trail.values()))

    abs_curve = []
    for t in arr_index:
        still_missing = (fvi_lead_vals > t).sum() + (fvi_trail_vals > (len(arr_index)-1-t)).sum()
        abs_curve.append(still_missing / len(lead_cols))

    rel_curve = []
    denom = X_full.shape[1]
    for t in arr_index:
        still_missing = (fvi_lead_vals > t).sum() + (fvi_trail_vals > (len(arr_index)-1-t)).sum()
        rel_curve.append(still_missing / denom)

    plt.figure()
    plt.plot(X_full.index, abs_curve)
    plt.xlabel("Year")
    plt.ylabel("Share")
    plt.title("Leading & trailing NAs: Share missing within leading-NA group")
    plt.tight_layout()

    save_path1 = OUT_DIR / "leading_trailing_NA_share_abs.png"
    plt.savefig(save_path1, dpi=300, bbox_inches='tight')
    print(f"Saved to: {save_path1}")

    plt.show()

    plt.figure()
    plt.plot(X_full.index, rel_curve)
    plt.xlabel("Year")
    plt.ylabel("Share (relative to all columns)")
    plt.title("Leading & trailing NAs: Share missing within all time series")
    plt.tight_layout()

    save_path2 = OUT_DIR / "leading_trailing_NA_share_rel.png"
    plt.savefig(save_path2, dpi=300, bbox_inches='tight')
    print(f"Saved to: {save_path2}")

    plt.show()

jump_counts_lead = pd.Series(fvi_lead_vals).value_counts().sort_index()
for pos, count in jump_counts_lead.items():
    print(f"{count} series start at {X_full.index[pos].date()}")

jump_counts_trail = pd.Series(fvi_trail_vals).value_counts().sort_index()
for pos, count in jump_counts_trail.items():
    print(f"{count} series end at {X_full.index[-pos-1].date()}")

## 14) Target plots: IP level and MoM change

In [ ]:
plt.figure()
plt.plot(y_level.index, y_level.values, color="black")
plt.title("Industrial Production")
plt.xlabel("Year")
plt.ylabel("Index")
plt.tight_layout()

save_path1 = OUT_DIR / "industrial_production.png"
plt.savefig(save_path1, dpi=300, bbox_inches='tight')
print(f"Saved to: {save_path1}")

plt.show()

rm = y_change.rolling(rm_window, min_periods=1).mean()
plt.figure()
plt.scatter(y_change.index, y_change.values, s=8, color="blue")
plt.plot(rm.index, rm.values, color="black")
plt.title("IP – Month over Month Change (Dots) + 12-Month Running Mean (Line)")
plt.xlabel("Year")
plt.ylabel("MoM-Change")
plt.tight_layout()

save_path2 = OUT_DIR / "ip_mom_change_running_mean.png"
plt.savefig(save_path2, dpi=300, bbox_inches='tight')
print(f"Saved to: {save_path2}")

plt.show()

## 15) Feature–target correlations by question (lags 0–12)

In [ ]:
def corr_lag(x, y, lag, min_obs=24):
    xs = x.shift(lag)
    df = pd.concat([xs, y], axis=1).dropna()
    if df.shape[0] < min_obs:
        return np.nan
    return float(np.corrcoef(df.iloc[:, 0], df.iloc[:, 1])[0, 1])

lags = range(0, 13)
cand = {}
for c in X_full.columns:
    _, qn = split_norm(c)
    if qn in QUESTION_CANON:
        cand.setdefault(qn, []).append(c)

mats = {}
for qn, cols in cand.items():
    if not cols:
        continue
    M = np.full((len(lags), len(cols)), np.nan)
    for j, col in enumerate(cols):
        x = X_full[col].astype(float)
        for i, lag in enumerate(lags):
            M[i, j] = corr_lag(x, y_change, lag)
    mats[qn] = pd.DataFrame(M, index=[str(l) for l in lags], columns=cols)

records = []
for qn, M in mats.items():
    for lag in M.index:
        vals = M.loc[lag].dropna().astype(float)
        for v in vals:
            records.append({"question": qn, "corr": v})
df_corrs = pd.DataFrame(records)
if df_corrs.empty:
    raise ValueError("No correlations calculated.")

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_corrs, x="question", y="corr", color="lightsteelblue")
plt.axhline(0, color="k", lw=1)
plt.title("Distribution of Correlations per Question (across all Lags & Industries)")
plt.xlabel("Question (QUESTION_CANON)")
plt.ylabel("Correlation Coefficient (ΔIP)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

save_path1 = OUT_DIR / "corr_distribution_per_question_boxplot.png"
plt.savefig(save_path1, dpi=300, bbox_inches='tight')
print(f"Saved to: {save_path1}")

plt.show()

plt.figure(figsize=(10, 6))
sns.histplot(data=df_corrs, x="corr", hue="question", element="step", stat="density", common_norm=False)
plt.axvline(0, color="k", lw=1)
plt.title("Histogram of Correlations per Question")
plt.xlabel("Correlation Coefficient (ΔIP)")
plt.ylabel("Density")
plt.tight_layout()

save_path2 = OUT_DIR / "corr_distribution_per_question_hist.png"
plt.savefig(save_path2, dpi=300, bbox_inches='tight')
print(f"Saved to: {save_path2}")

plt.show()

## 16) Best (absolute) correlation per series (from mats)

In [ ]:
corr_values = []
for qn, M in mats.items():
    for col in M.columns:
        series_corrs = M[col].astype(float)
        if series_corrs.isna().all():
            continue
        best_corr = series_corrs.loc[series_corrs.abs().idxmax()]
        corr_values.append(best_corr)

corr_values = pd.Series(corr_values, name="best_corr")

plt.figure(figsize=(8, 5))
sns.histplot(corr_values, bins=40, kde=True, color="steelblue")
plt.title("Distribution of Highest Correlations per Time Series (Lags 0–12)")
plt.xlabel("Correlation Coefficient (ΔIP)")
plt.ylabel("Count of Time Series")
plt.axvline(0, color="k", lw=1)
plt.tight_layout()

save_path1 = OUT_DIR / "best_correlation_distribution.png"
plt.savefig(save_path1, dpi=300, bbox_inches='tight')
print(f"Saved to: {save_path1}")

plt.show()

plt.figure(figsize=(8, 5))
sns.histplot(corr_values.abs(), bins=40, kde=True, color="darkorange")
plt.title("Distribution of Highest Absolute Correlations per Time Series (Lags 0–12)")
plt.xlabel("|Correlation Coefficient|")
plt.ylabel("Count of Time Series")
plt.tight_layout()

save_path2 = OUT_DIR / "best_abs_correlation_distribution.png"
plt.savefig(save_path2, dpi=300, bbox_inches='tight')
print(f"Saved to: {save_path2}")

plt.show()

## 17) Correlations by lag (lags 1–6) for NA-free features

In [ ]:
complete_feats = X_full.columns[X_full.notna().all(axis=0)]
print(f"NA-free features: {len(complete_feats)} / {X_full.shape[1]}")

mats_complete = {}
complete_set = set(complete_feats)
for qn, M in mats.items():
    keep_cols = [c for c in M.columns if c in complete_set]
    if keep_cols:
        mats_complete[qn] = M[keep_cols]

records = []
for qn, M in mats_complete.items():
    for lag in M.index:
        lag_int = int(lag)
        if not (1 <= lag_int <= 6):
            continue
        vals = M.loc[lag].dropna().astype(float).values
        records.extend([{"lag": lag_int, "corr": v} for v in vals])

df_corrs = pd.DataFrame(records)
if df_corrs.empty:
    raise ValueError("No correlations left — likely no completely NA-free feature series in mats.")

df_corrs["lag"] = pd.Categorical(df_corrs["lag"], categories=list(range(1, 7)), ordered=True)

plt.figure(figsize=(8, 5))
sns.boxplot(data=df_corrs, x="lag", y="corr", color="lightsteelblue")
plt.axhline(0, color="k", lw=1)

plt.title("Correlation between IP & ifo features by lag")
plt.xlabel("Lag (months)")
plt.ylabel("Pearson correlation coefficient")
plt.tight_layout()

save_path1 = OUT_DIR / "correlation_distribution_per_lag_nona_features.png"
plt.savefig(save_path1, dpi=300, bbox_inches="tight")
print(f"Saved to: {save_path1}")
plt.show()

df_abs = df_corrs.assign(abs_corr=lambda d: d["corr"].abs())
lag_summary = df_abs.groupby("lag")["abs_corr"].median().reset_index()

plt.figure(figsize=(8, 4))
sns.lineplot(data=lag_summary, x="lag", y="abs_corr", marker="o")

plt.title("Median absolute feature–target correlation by lag")
plt.xlabel("Lag (months)")
plt.ylabel("Median absolute Pearson correlation")
plt.tight_layout()

save_path2 = OUT_DIR / "median_abs_correlation_per_lag_nona_features.png"
plt.savefig(save_path2, dpi=300, bbox_inches="tight")
print(f"Saved to: {save_path2}")
plt.show()

## 18) Sanity summary for lag 1–6 correlation sample

In [ ]:
corr = df_corrs["corr"].astype(float).to_numpy()

max_corr = np.nanmax(corr)
min_corr = np.nanmin(corr)
max_abs_corr = np.nanmax(np.abs(corr))
pct_abs_lt_01 = 100 * np.mean(np.abs(corr) < 0.1)

print("Summary for BOXPlot data only (lags 1–6)")
print(f"N = {len(corr):,}")
print(f"Max corr            = {max_corr:.4f}")
print(f"Min corr            = {min_corr:.4f}")
print(f"Max |corr|          = {max_abs_corr:.4f}")
print(f"% with |corr| < 0.1 = {pct_abs_lt_01:.2f}%")

i_max = int(np.nanargmax(corr))
i_abs = int(np.nanargmax(np.abs(corr)))
print(f"Max corr occurs at lag={df_corrs.iloc[i_max]['lag']}, corr={df_corrs.iloc[i_max]['corr']:.4f}")
print(f"Max |corr| occurs at lag={df_corrs.iloc[i_abs]['lag']}, corr={df_corrs.iloc[i_abs]['corr']:.4f}")

print(f"Any corr > 0.2? {bool(np.any(corr > 0.2))}")
print(f"Any |corr| > 0.2? {bool(np.any(np.abs(corr) > 0.2))}")

## 20) Autocorrelation of ΔIP (lags 1–6) + Ljung–Box

In [ ]:
y_change = df["IP_change"].dropna()

lags = range(1, 7)
vals = []
for k in lags:
    a = pd.concat([y_change.shift(k), y_change], axis=1).dropna()
    if len(a) < 3:
        vals.append(np.nan)
    else:
        vals.append(float(np.corrcoef(a.iloc[:, 0], a.iloc[:, 1])[0, 1]))

plt.figure()
plt.stem(list(lags), vals)
plt.title("Autocorrelation ΔIP (1..6)")
plt.xlabel("Lag")
plt.ylabel("ACF")
plt.tight_layout()

save_path1 = OUT_DIR / "autocorrelation_delta_ip_1-6.png"
plt.savefig(save_path1, dpi=300, bbox_inches="tight")
print(f"Saved to: {save_path1}")
plt.show()

n = len(y_change)
conf = 1.96 / np.sqrt(n)

plt.figure()
plt.stem(list(lags), vals)
plt.axhline(y=0, color="black", linewidth=1)
plt.axhline(y=conf, color="red", linestyle="--")
plt.axhline(y=-conf, color="red", linestyle="--")
plt.title("Autocorrelation ΔIP (1..6) with 95% band (white-noise approx.)")
plt.xlabel("Lag")
plt.ylabel("ACF")
plt.tight_layout()

save_path2 = OUT_DIR / "autocorrelation_delta_ip_confband_1-6.png"
plt.savefig(save_path2, dpi=300, bbox_inches="tight")
print(f"Saved to: {save_path2}")
plt.show()

lb = acorr_ljungbox(y_change, lags=list(lags), return_df=True)
print("\nLjung–Box test (H0: no autocorrelation up to lag h):")
print(lb)